# Scraping FOMC Policy Statements

In [213]:
import requests
from bs4 import BeautifulSoup
import bs4
import random
import time
import pandas as pd
import re
import numpy as np

I imitate a request from a normal user by taking the headers from one of my requests to the Fed website and using these in my subsequent requests. For whatever reason, my normal way of grabbing request headers did not work, but the version I got from [curlconverter.com](https://curlconverter.com/) seemed to work fine.

In [2]:
cookies = {
    'JSESSIONID': '3BEF2ACFDAA5C6A04A42198CA8E30A44',
    'ak_bmsc': 'E8CCCEB6062513F0289F84F4C0AC2A30~000000000000000000000000000000~YAAQxh/XF4DVSpiKAQAAndeFuRXWl6BZ43njAtjeko96aLJCSlOQcL+hjKC0K+Zp4WBwHxqgUcASS3+2/hmVzotCS9gKN3OP/KFfWbhtYhySxbcnKCZxuRUrmWhQeISO9sWACxxUkrbX4MUnVa64RKJqELkuMB4KY2euSnJ+UqrZTfm038LAPx9gFiNeLDxY8G7NeskPdBQDQFGM+a513Igxog+mP78sxWqB3XfC3DPzZ5R+5MWrt/pSX3+hEg3AOHtR2KULGZxzXobCY+g90+1rtLmLwlpsk4Z0v/p9xo2sZGaXtxzOgiZ1G5LIuKAVHaPPH+wISJBIKlDV6IAQ1/3v9fRAoOL8WzVElDtAxJOGnyxM9HPUPFGkikNemAkFj9C5irHbKNiL17ucxcbU',
    'bm_sv': '1F00644910C6E626FD359D1B33690F6B~YAAQ1mrcF4eod5CKAQAAnyOvuRVFZ516Gq+KLXMdGVaehSXqBISu+cS0nofU6GQcJYx3ZxCnEtJyrBNbZIOz5ywU3TTNt8RgFrYnMlLkQveQxe7pEUmjh/S1ksPOc5bkRaUk72hFkbNz4Y/LYRFMDn9vCZt3V9ZgR9NSfFSiyCtf2eTeTtwQagK0An/C841QN1AEe6kCukrHHZnQhJMjfjyMjUHQHvYznJTzkpe0/W0Czo6Wcg9VBZLvuaGoyU5vZgzY+A==~1',
    'cookiesession1': '678A3E44D8504EE07ECF240D79740297',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.fedsearch.org/fomc-docs/search?advanced_search=true&from_year=1998&search_precision=All+Words&start=180&sort=Most+Recent+First&to_month=9&to_year=2014&number=10&fomc_document_type=policystatement&Search=Search&text=&from_month=3',
    'Connection': 'keep-alive',
    # 'Cookie': 'JSESSIONID=3BEF2ACFDAA5C6A04A42198CA8E30A44; ak_bmsc=E8CCCEB6062513F0289F84F4C0AC2A30~000000000000000000000000000000~YAAQxh/XF4DVSpiKAQAAndeFuRXWl6BZ43njAtjeko96aLJCSlOQcL+hjKC0K+Zp4WBwHxqgUcASS3+2/hmVzotCS9gKN3OP/KFfWbhtYhySxbcnKCZxuRUrmWhQeISO9sWACxxUkrbX4MUnVa64RKJqELkuMB4KY2euSnJ+UqrZTfm038LAPx9gFiNeLDxY8G7NeskPdBQDQFGM+a513Igxog+mP78sxWqB3XfC3DPzZ5R+5MWrt/pSX3+hEg3AOHtR2KULGZxzXobCY+g90+1rtLmLwlpsk4Z0v/p9xo2sZGaXtxzOgiZ1G5LIuKAVHaPPH+wISJBIKlDV6IAQ1/3v9fRAoOL8WzVElDtAxJOGnyxM9HPUPFGkikNemAkFj9C5irHbKNiL17ucxcbU; bm_sv=1F00644910C6E626FD359D1B33690F6B~YAAQ1mrcF4eod5CKAQAAnyOvuRVFZ516Gq+KLXMdGVaehSXqBISu+cS0nofU6GQcJYx3ZxCnEtJyrBNbZIOz5ywU3TTNt8RgFrYnMlLkQveQxe7pEUmjh/S1ksPOc5bkRaUk72hFkbNz4Y/LYRFMDn9vCZt3V9ZgR9NSfFSiyCtf2eTeTtwQagK0An/C841QN1AEe6kCukrHHZnQhJMjfjyMjUHQHvYznJTzkpe0/W0Czo6Wcg9VBZLvuaGoyU5vZgzY+A==~1; cookiesession1=678A3E44D8504EE07ECF240D79740297',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

Now, I once again rely on CurlConverter to get the request parameters from the page, keeping the year restriction parameters and varying the `start` parameter to get links to policy statements in groups of 10. I use BeautifulSoup to find the elements of interest: a "type" field which happens to contain the date, and a separate anchor tag that contains a link to the speech.

In [ ]:

def fetch_speeches(offset):
    params = {
        'advanced_search': 'true',
        'from_year': '1998',
        'search_precision': 'All Words',
        'start': str(offset),
        'sort': 'Most Recent First',
        'to_month': '9',
        'to_year': '2014',
        'number': '10',
        'fomc_document_type': 'policystatement',
        'Search': 'Search',
        'text': '',
        'from_month': '3',
    }

    response = requests.get('https://www.fedsearch.org/fomc-docs/search', params=params, cookies=cookies, headers=headers)
    soup = BeautifulSoup(response.text)

    
    speeches = soup.find('div', id='pagenav').find_all_next('strong')

    urls = []
    for speech in speeches:
        record = {}
        if speech.find('a') is None:
            continue
        url = speech.find('a')['href']
        urls.append(url)

    type_elems = soup.find('div', id='pagenav').find_all_next('span', {'class': 'greentext'})

    types = []
    for type_elem in type_elems:
        oneline = type_elem.text.replace('\t', '').replace('\r', '').replace('\n', '')
        type_text = re.search('Type: (.*) \(', oneline).group(1).strip()
        types.append(type_text)

    records = []
    for url, type_elem in zip(urls, types):
        records.append({'url': url, 'title': type_elem})
    return records

Here I test the function above to make sure it is working correctly.

In [3]:
fetch_speeches(170)

[{'url': 'https://www.federalreserve.gov/boarddocs/press/general/1999/19990824/',
  'title': 'Policy Statement on 08/24/1999'},
 {'url': 'https://www.federalreserve.gov/boarddocs/press/general/1999/19990824/default.htm',
  'title': 'Policy Statement on 08/24/1999'},
 {'url': 'https://www.federalreserve.gov/boarddocs/press/general/1999/19990630/',
  'title': 'Policy Statement on 06/30/1999'},
 {'url': 'https://www.federalreserve.gov/boarddocs/press/general/1999/19990630/default.htm',
  'title': 'Policy Statement on 06/30/1999'},
 {'url': 'https://www.federalreserve.gov/boarddocs/press/general/1999/19990518/',
  'title': 'Policy Statement on 05/18/1999'},
 {'url': 'https://www.federalreserve.gov/boarddocs/press/general/1998/19981117/',
  'title': 'Policy Statement on 11/17/1998'},
 {'url': 'https://www.federalreserve.gov/boarddocs/press/general/1998/19981117/default.htm',
  'title': 'Policy Statement on 11/17/1998'},
 {'url': 'https://www.federalreserve.gov/boarddocs/press/general/1998/1

Now, I repeat this over 19 times to get all 181 policy statements (there are some that are posted multiple times, which I deal with later). As always, I make sure to put a delay between requests to be courteous to the website administrators.

In [ ]:
records = []

for i in range(19):
    time.sleep(random.random() * 5  + 5)
    print(i)
    records = records + fetch_speeches(i * 10)

Next, I start exploring the actual links to the speeches to see how many different formats I must account for. Thankfully, there are relatively few.

I first tackled the format used most recently, using BeautifulSoup to locate the relevant text.

In [349]:
def get_format_1(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    paras = soup.find('div', {'id': 'article'}).find('div', {'class': 'col-xs-12 col-sm-8 col-md-8'}).find_all('p') #, {'class': None}).find_all('p')
    # print(paras)
    para_text = []
    for para in paras:
        if para.find('a') is None :
            para_text = para_text + [para.text.strip()]
    return para_text

A few test cases of speeches published in this format.

In [332]:
# get_format_1('https://www.federalreserve.gov/newsevents/pressreleases/monetary20090923a.htm')
get_format_1('https://www.federalreserve.gov/newsevents/pressreleases/monetary20140730a.htm')
# get_format_1('https://www.federalreserve.gov/newsevents/pressreleases/monetary20080916a.htm')

["Information received since the Federal Open Market Committee met in June indicates that growth in economic activity rebounded in the second quarter. Labor market conditions improved, with the unemployment rate declining further. However, a range of labor market indicators suggests that there remains significant underutilization of labor resources. Household spending appears to be rising moderately and business fixed investment is advancing, while the recovery in the housing sector remains slow. Fiscal policy is restraining economic growth, although the extent of restraint is diminishing. Inflation has moved somewhat closer to the Committee's longer-run objective. Longer-term inflation expectations have remained stable.",
 'Consistent with its statutory mandate, the Committee seeks to foster maximum employment and price stability. The Committee expects that, with appropriate policy accommodation, economic activity will expand at a moderate pace, with labor market indicators and inflat

Here's another test case for some of the older speeches. It turned out that I had to create three separate formats for the older speeches, as there were more frequent changes to how the speeches were formatted in these earlier years.

In [348]:
def get_format_2(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    elems = soup.find('table', attrs={'width': '600'})
    paras = elems.find('p').find('p').find('p').find_all('p')
    para_text = []
    for elem in paras:
        if len(re.findall('[0-9]{4} Monetary policy', elem.text)) > 0:
            break
        para_text.append(elem.text.strip())
    return para_text

In [331]:
get_format_2('https://www.federalreserve.gov/boarddocs/press/monetary/2005/20050322/default.htm')


['The Federal Open Market Committee decided today to raise its target for the federal funds rate by 25 basis points to 2-3/4 percent.',
 'The Committee believes that, even after this action, the stance of monetary policy remains accommodative and, coupled with robust underlying growth in productivity, is providing ongoing support to economic activity.  Output evidently continues to grow at a solid pace despite the rise in energy prices, and labor market conditions continue to improve gradually.  Though longer-term inflation expectations remain well contained, pressures on inflation have picked up in recent months and pricing power is more evident.  The rise in energy prices, however, has not notably fed through to core consumer prices.',
 'The Committee perceives that, with appropriate monetary policy action, the upside and downside risks to the attainment of both sustainable growth and price stability should be kept roughly equal.  With underlying inflation expected to be contained, t

In [347]:
# for use on and before aug 24 1999

def get_format_3(url):
    response = requests.get(url, 'lxml')
    soup = BeautifulSoup(response.text)
    td = soup.find('table', {'width': '600'}).find('td')
    para_text = []
    for elem in td.children:
        if isinstance(elem, bs4.element.NavigableString):
            para_text += [elem.text.strip()]
        else:
            for child in elem.children:
                para_text += child.text.strip().split('\r\n\r\n\t')
        
    return para_text

In [328]:
get_format_3('https://www.federalreserve.gov/boarddocs/press/general/1998/19980929/default.htm')
# get_format_3('https://www.federalreserve.gov/boarddocs/press/general/1999/19990824/')

i
i


['The Federal Open Market Committee decided today to ease the stance of monetary policy slightly, expecting the federal funds rate to decline 1/4 percentage point to around 5-1/4 percent.',
 'The action was taken to cushion the effects on prospective economic growth in the United States of increasing weakness in foreign economies and of less accommodative financial conditions domestically.  The recent changes in the global economy and adjustments in U.S. financial markets mean that a slightly lower federal funds rate should now be consistent with keeping inflation low and sustaining economic growth going forward.',
 'The discount rate remains unchanged at 5 percent.']

In [346]:

def get_format_4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # print(soup)
    td = soup.find('table', {'width': '600'}).find('td')
    para_text = []
    para_text = [sent.strip() for sent in td.text.split('\r\n\r') if sent.strip() != '']
        
    return para_text

In [326]:
get_format_4('https://www.federalreserve.gov/boarddocs/press/general/2001/20010320/')
get_format_4('https://www.federalreserve.gov/boarddocs/press/general/2000/20000628/')

['The Federal Open Market Committee at its meeting today decided to lower its target for the federal funds rate by 50 basis points to 5 percent. In a related action, the Board of Governors approved a 50 basis point reduction in the discount rate to 4-1/2 percent.',
 'Persistent pressures on profit margins are restraining investment spending and, through declines in equity wealth, consumption. The associated backup in inventories has induced a rapid response in manufacturing output and, with spending having firmed a bit since last year, inventory adjustment appears to be well underway.',
 'Although current developments do not appear to have materially diminished the prospects for long-term growth in productivity, excess productive capacity has emerged recently.  The possibility that this excess could continue for some time and the potential for weakness in global economic conditions suggest substantial risks that demand and production could remain soft.  In these circumstances, when the

Now, I make a dataframe from the existing records which I will use to eventually store the full text versions.

In [ ]:
df = pd.DataFrame.from_records(records)
# declare the text field as an object so I can store arrays of paragraphs to preserve structure
df['text'] = None
df['text'] = df['text'].astype('object')

Next, I iterate through this dataframe, using the dates of format changes as landmarks to decide which format to use when extracting text from the page.

In [ ]:
for index, row in df.iterrows():
    print(index)
    time.sleep(random.random() * 4 + 4)
    if index <= 80:
        df['text'][index] = get_format_1(row[0])
    elif index > 80 and index <= 92:
        df['text'][index] = get_format_2(row[0])
    elif index > 92 and index < 170:
        df['text'][index] = get_format_4(row[0])
    else:
        df['text'][index] = get_format_3(row[0])

Finally, I store this as a csv

In [383]:
df.to_csv('statements.csv')